## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

In [5]:
import numpy as np
import pandas as pd
import random
import string

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [7]:
a=np.random.randint(0, 1001, size=1000000)
b=np.array([x+100 for x in a])
print(a)
print(b)
sr=np.mean(b)
print(sr)

[769 324 363 ... 848 432 349]
[869 424 463 ... 948 532 449]
599.855797


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [114]:
key=[''.join(random.choices(string.ascii_letters, k=5)) for _ in range(2000000)]
table=pd.DataFrame(np.random.randint(0, 1001, size=(2000000,4)), index=key)
table

,0,1,2,3
ZkBYY,100,411,810,112
hOpSI,678,766,819,731
lMrwS,50,528,346,260
yGkwv,5,404,86,81
TywEa,356,687,978,266
...,...,...,...,...
MJPqy,892,67,197,662
VUyNG,894,970,781,265
spLnE,809,638,863,714
HpwsA,909,462,497,907


In [116]:
table.loc['spLnE']

0    809
1    638
2    863
3    714
Name: spLnE, dtype: int32

In [115]:
try:
    table.loc['abcde']
except KeyError:
    print('нет key, где указаны первые 5 английских букв')

нет key, где указаны первые 5 английских букв


## Лабораторная работа 3

In [1]:
!pip install line_profiler

In [94]:
%load_ext line_profiler

In [4]:
import timeit

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [6]:
recipes = pd.read_csv('recipes_sample.csv', delimiter=',')
reviews = pd.read_csv('reviews_sample.csv', delimiter=',')
reviews.rename(columns = {'Unnamed: 0':'id'}, inplace = True )
df=pd.merge(recipes, reviews)
df['submitted'] = pd.to_datetime(df['submitted'])

In [7]:
df

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,user_id,recipe_id,date,rating,review
0,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN,2000576480,445211,2015-10-18,1,Terrible - makes a very runny batter like for ...
1,say what banana sandwich,95926,5,118163,2004-07-20,4.0,you just have to try it to believe it.,NaN,221694,126623,2009-02-01,4,This was yummy. I cut the entire sauce recipe ...
2,add in anything muffins,149593,15,89831,2005-12-28,NaN,"this is a never-fail muffin recipe, it's a bla...",9.0,52282,66815,2008-07-12,5,"my family all enjoyed this dish, and yes it do..."
3,burek or feta cheese phyllo pie,310570,65,676820,2008-06-24,38.0,"ok, there are different version of burek (some...",6.0,527607,260529,2007-10-23,5,These were fantastic. I lightened it even fur...
4,skordy new potatoes w rosemary lemon olive oi,296983,35,718054,2008-04-08,NaN,i took this recipe from a vegan tastes of gree...,6.0,795795,50385,2008-03-20,5,This recipe was WONDERFULLY DELICIOUS! The rev...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3446,zucchini meat sauce with pasta,253264,30,275742,2007-09-17,NaN,havn't tried this recipe yet but hopefully wil...,6.0,142849,102617,2007-08-01,5,This is fabulous! I do cheat however and use t...
3447,zucchini stuffed with feta and basil,72857,30,95743,2003-10-08,NaN,"easy to make, savory and a nice presentation. ...",5.0,49937,54269,2003-04-20,5,This cake mix extender is great! I used it wit...
3448,zucchini tomato salsa,321224,20,464080,2008-08-25,4.0,a contest winner from taste of home. it state...,13.0,171790,394467,2010-01-11,5,I loved this soup! The only changes I made wer...
3449,zucchini potato and herb fritters,344542,30,197023,2008-12-21,8.0,adapted from a recipe in 'the australian women...,12.0,659599,222188,2013-10-15,5,So simple but yet so delicious. I only made h...


In [14]:
#A
def average():
    total_rating = 0
    count = 0
    for index, row in df.iterrows():
        if row['submitted'].year >= 2010:
            total_rating += row['rating']
            count += 1
            average_rating = total_rating / count
    return average_rating
%prun av=average() 
print(av)
execution_time=timeit.timeit(average, number=1)
print(f'Время исполнения - {execution_time}')

 4.371495327102804
Время исполнения - 0.14723639999999705


In [17]:
af2010 = df.loc[df['submitted'].dt.year >= 2010]

In [18]:
#Б

def average1():
    total_rating = 0
    count = 0
    for index, row in af2010.iterrows():
        total_rating += row['rating']
        count += 1
        average_rating = total_rating / count
    return average_rating

%prun av1=average1()
print(av1)
execution_time=timeit.timeit(average1, number=1)
print(f'Время исполнения - {execution_time}')

 4.371495327102804
Время исполнения - 0.02514939999997523


In [19]:
#В
def average2():
    return af2010['rating'].mean()

%prun av2=average2()
print(av2)
execution_time=timeit.timeit(average2, number=1)
print(f'Время исполнения - {execution_time}')

 4.371495327102804
Время исполнения - 0.0001850000001013541


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

***1 вариант***

 447942 function calls (441036 primitive calls) in 0.265 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     3451    0.025    0.000    0.199    0.000 series.py:323(__init__)
   109169    0.014    0.000    0.018    0.000 {built-in method builtins.isinstance}
     3451    0.014    0.000    0.056    0.000 construction.py:470(sanitize_array)
     3451    0.010    0.000    0.016    0.000 cast.py:1466(maybe_infer_to_datetimelike)
     3879    0.009    0.000    0.048    0.000 series.py:943(__getitem__)
     3451    0.008    0.000    0.029    0.000 blocks.py:2041(new_block)
     3452    0.008    0.000    0.026    0.000 generic.py:5577(__setattr__)
     3451    0.007    0.000    0.009    0.000 generic.py:239(__init__)
41446/34540    0.006    0.000    0.009    0.000 {built-in method builtins.len}
        1    0.006    0.006    0.265    0.265 934095590.py:2(average)
     3452    0.006    0.000    0.211    0.000 frame.py:1279(iterrows)
     3451    0.006    0.000    0.022    0.000 construction.py:695(_try_cast)
     3451    0.006    0.000    0.015    0.000 config.py:109(_get_single_key)
     3451    0.006    0.000    0.039    0.000 managers.py:1731(from_array)
     3879    0.006    0.000    0.031    0.000 series.py:1052(_get_value)
     3451    0.005    0.000    0.006    0.000 blocks.py:1989(get_block_type)
     3451    0.004    0.000    0.008    0.000 blocks.py:1960(maybe_coerce_values)
     6902    0.004    0.000    0.004    0.000 config.py:603(_get_deprecated_option)
     3451    0.004    0.000    0.012    0.000 series.py:640(name)
     3879    0.004    0.000    0.005    0.000 managers.py:1848(internal_values)
     3451    0.004    0.000    0.005    0.000 config.py:589(_get_root)
     3879    0.004    0.000    0.011    0.000 base.py:5646(_get_values_for_loc)
     3879    0.004    0.000    0.014    0.000 base.py:3577(get_loc)
     3451    0.004    0.000    0.006    0.000 blocks.py:2055(check_ndim)
     3451    0.004    0.000    0.004    0.000 {pandas._libs.lib.infer_datetimelike_array}
        1    0.004    0.004    0.004    0.004 {pandas._libs.tslibs.vectorized.ints_to_pydatetime}
     3451    0.004    0.000    0.006    0.000 series.py:590(name)
    10360    0.003    0.000    0.005    0.000 generic.py:43(_check)
     3879    0.003    0.000    0.003    0.000 {method 'get_loc' of 'pandas._libs.index.IndexEngine' objects}
     3451    0.003    0.000    0.023    0.000 config.py:127(_get_option)
     3879    0.003    0.000    0.005    0.000 indexing.py:2481(check_deprecated_indexers)
     3879    0.003    0.000    0.007    0.000 base.py:6284(_maybe_cast_indexer)
     3451    0.003    0.000    0.003    0.000 generic.py:5561(__getattr__)
     3451    0.003    0.000    0.005    0.000 construction.py:627(_sanitize_ndim)
     6903    0.002    0.000    0.003    0.000 inference.py:321(is_hashable)
     3451    0.002    0.000    0.008    0.000 common.py:1721(validate_all_hashable)
     3451    0.002    0.000    0.004    0.000 construction.py:802(is_empty_data)
     6906    0.002    0.000    0.003    0.000 base.py:884(__len__)
     3453    0.002    0.000    0.006    0.000 {built-in method builtins.all}
     6902    0.002    0.000    0.004    0.000 common.py:1740(<genexpr>)
     3451    0.002    0.000    0.025    0.000 config.py:255(__call__)
     3451    0.002    0.000    0.002    0.000 flags.py:47(__init__)
     3451    0.002    0.000    0.002    0.000 construction.py:438(ensure_wrapped_if_datetimelike)
     3451    0.002    0.000    0.002    0.000 managers.py:1700(__init__)
     3879    0.002    0.000    0.007    0.000 series.py:687(_values)
     3451    0.002    0.000    0.010    0.000 construction.py:379(extract_array)
    13815    0.002    0.000    0.002    0.000 {built-in method builtins.getattr}
     3451    0.002    0.000    0.003    0.000 base.py:7082(maybe_extract_name)
     3879    0.002    0.000    0.002    0.000 base.py:2344(is_floating)
    10353    0.002    0.000    0.002    0.000 {pandas._libs.lib.is_list_like}
     3451    0.002    0.000    0.004    0.000 config.py:642(_warn_if_deprecated)
     3451    0.002    0.000    0.002    0.000 series.py:542(_set_axis)
     3451    0.002    0.000    0.002    0.000 construction.py:664(_sanitize_str_dtypes)
     3451    0.002    0.000    0.004    0.000 common.py:552(require_length_match)
     3879    0.001    0.000    0.002    0.000 common.py:160(cast_scalar_indexer)
     3451    0.001    0.000    0.002    0.000 construction.py:684(_maybe_repeat)
     3451    0.001    0.000    0.003    0.000 config.py:630(_translate_key)
     3879    0.001    0.000    0.002    0.000 common.py:346(apply_if_callable)
     3451    0.001    0.000    0.001    0.000 config.py:571(_select_options)
     3451    0.001    0.000    0.001    0.000 {built-in method numpy.array}
     3451    0.001    0.000    0.001    0.000 base.py:6987(ensure_index)
     3451    0.001    0.000    0.001    0.000 {method 'split' of 'str' objects}
     3451    0.001    0.000    0.001    0.000 common.py:1416(is_1d_only_ea_dtype)
     6903    0.001    0.000    0.001    0.000 {built-in method builtins.hash}
     6908    0.001    0.000    0.001    0.000 {built-in method builtins.issubclass}
     7758    0.001    0.000    0.001    0.000 {pandas._libs.lib.is_integer}
     3451    0.001    0.000    0.001    0.000 managers.py:1792(_block)
     6902    0.001    0.000    0.001    0.000 typing.py:1375(cast)
        3    0.001    0.000    0.001    0.000 {method 'astype' of 'numpy.ndarray' objects}
     3451    0.001    0.000    0.001    0.000 {built-in method builtins.hasattr}
        1    0.001    0.001    0.006    0.006 managers.py:1601(_interleave)
     3879    0.001    0.000    0.001    0.000 {pandas._libs.lib.is_scalar}
     3879    0.001    0.000    0.001    0.000 {built-in method builtins.callable}
     3879    0.001    0.000    0.001    0.000 {pandas._libs.lib.is_float}
     3451    0.000    0.000    0.000    0.000 {pandas._libs.algos.ensure_object}
        1    0.000    0.000    0.265    0.265 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {built-in method numpy.empty}
        1    0.000    0.000    0.006    0.006 managers.py:1541(as_array)
        1    0.000    0.000    0.265    0.265 {built-in method builtins.exec}
        2    0.000    0.000    0.000    0.000 numerictypes.py:575(_can_coerce_all)
        1    0.000    0.000    0.004    0.004 datetimelike.py:403(astype)
       19    0.000    0.000    0.000    0.000 numerictypes.py:584(<listcomp>)
        1    0.000    0.000    0.000    0.000 cast.py:1789(find_common_type)
        1    0.000    0.000    0.004    0.004 datetimes.py:645(astype)
        1    0.000    0.000    0.000    0.000 base.py:744(__iter__)
        1    0.000    0.000    0.004    0.004 blocks.py:1461(get_values)
        2    0.000    0.000    0.000    0.000 common.py:1747(pandas_dtype)
        4    0.000    0.000    0.000    0.000 common.py:1274(is_bool_dtype)
        3    0.000    0.000    0.001    0.000 blocks.py:222(get_values)
        2    0.000    0.000    0.000    0.000 {method 'reshape' of 'numpy.ndarray' objects}
        2    0.000    0.000    0.000    0.000 base.py:53(shape)
        9    0.000    0.000    0.000    0.000 common.py:1552(get_dtype)
        1    0.000    0.000    0.000    0.000 managers.py:1615(<listcomp>)
        1    0.000    0.000    0.000    0.000 generic.py:5632(_protect_consolidate)
        2    0.000    0.000    0.000    0.000 common.py:581(is_dtype_equal)
        4    0.000    0.000    0.000    0.000 datetimes.py:545(dtype)
        1    0.000    0.000    0.006    0.006 frame.py:10808(values)
        3    0.000    0.000    0.000    0.000 common.py:315(is_datetime64_dtype)
        1    0.000    0.000    0.000    0.000 {method 'view' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 datetimelike.py:280(asi8)
        1    0.000    0.000    0.000    0.000 dtypes.py:955(is_dtype)
        1    0.000    0.000    0.000    0.000 numerictypes.py:599(find_common_type)
        1    0.000    0.000    0.000    0.000 {pandas._libs.lib.dtypes_all_equal}
        1    0.000    0.000    0.000    0.000 generic.py:5650(f)
        5    0.000    0.000    0.000    0.000 cast.py:1835(<genexpr>)
        1    0.000    0.000    0.000    0.000 {built-in method numpy.zeros}
        6    0.000    0.000    0.000    0.000 base.py:55(<genexpr>)
        1    0.000    0.000    0.000    0.000 common.py:423(is_period_dtype)
        2    0.000    0.000    0.000    0.000 {built-in method builtins.any}
        1    0.000    0.000    0.000    0.000 common.py:161(is_object_dtype)
        1    0.000    0.000    0.000    0.000 common.py:925(is_datetime64_ns_dtype)
        3    0.000    0.000    0.000    0.000 cast.py:1828(<genexpr>)
        2    0.000    0.000    0.000    0.000 datetimes.py:570(tz)
        5    0.000    0.000    0.000    0.000 cast.py:1819(<genexpr>)
        4    0.000    0.000    0.000    0.000 blocks.py:244(mgr_locs)
        1    0.000    0.000    0.000    0.000 base.py:210(interleaved_dtype)
        1    0.000    0.000    0.000    0.000 {method 'transpose' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 base.py:286(is_dtype)
        1    0.000    0.000    0.000    0.000 numerictypes.py:651(<listcomp>)
        1    0.000    0.000    0.000    0.000 generic.py:5646(_consolidate_inplace)
        1    0.000    0.000    0.000    0.000 {built-in method fromkeys}
        1    0.000    0.000    0.000    0.000 base.py:465(find)
        3    0.000    0.000    0.000    0.000 base.py:4820(_values)
        1    0.000    0.000    0.000    0.000 common.py:1587(_is_dtype_type)
        1    0.000    0.000    0.000    0.000 blocks.py:350(shape)
        1    0.000    0.000    0.000    0.000 managers.py:1665(is_consolidated)
        1    0.000    0.000    0.000    0.000 managers.py:618(consolidate)
        1    0.000    0.000    0.000    0.000 managers.py:217(is_single_block)
        1    0.000    0.000    0.000    0.000 {method 'ravel' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 common.py:389(is_timedelta64_dtype)
        2    0.000    0.000    0.000    0.000 cast.py:1830(<genexpr>)
        1    0.000    0.000    0.000    0.000 common.py:145(classes)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 datetimelike.py:880(freq)
        1    0.000    0.000    0.000    0.000 {built-in method numpy.asarray}
        1    0.000    0.000    0.000    0.000 common.py:147(<lambda>)
        1    0.000    0.000    0.000    0.000 {method 'keys' of 'dict' objects}
        1    0.000    0.000    0.000    0.000 numerictypes.py:652(<listcomp>)

***2 вариант***

     54615 function calls (53755 primitive calls) in 0.036 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      428    0.003    0.000    0.027    0.000 series.py:323(__init__)
    13316    0.002    0.000    0.003    0.000 {built-in method builtins.isinstance}
      428    0.002    0.000    0.008    0.000 construction.py:470(sanitize_array)
      428    0.002    0.000    0.002    0.000 cast.py:1466(maybe_infer_to_datetimelike)
      428    0.001    0.000    0.006    0.000 series.py:943(__getitem__)
      428    0.001    0.000    0.004    0.000 blocks.py:2041(new_block)
      428    0.001    0.000    0.001    0.000 generic.py:239(__init__)
      429    0.001    0.000    0.004    0.000 generic.py:5577(__setattr__)
      429    0.001    0.000    0.029    0.000 frame.py:1279(iterrows)
        1    0.001    0.001    0.036    0.036 2647316429.py:3(average1)
5173/4313    0.001    0.000    0.001    0.000 {built-in method builtins.len}
      428    0.001    0.000    0.003    0.000 construction.py:695(_try_cast)
      428    0.001    0.000    0.002    0.000 config.py:109(_get_single_key)
      428    0.001    0.000    0.005    0.000 managers.py:1731(from_array)
      428    0.001    0.000    0.004    0.000 series.py:1052(_get_value)
      856    0.001    0.000    0.001    0.000 config.py:603(_get_deprecated_option)
      428    0.001    0.000    0.001    0.000 blocks.py:1989(get_block_type)
      428    0.001    0.000    0.001    0.000 blocks.py:1960(maybe_coerce_values)
      428    0.001    0.000    0.002    0.000 series.py:640(name)
      428    0.001    0.000    0.001    0.000 managers.py:1848(internal_values)
      428    0.001    0.000    0.001    0.000 {pandas._libs.lib.infer_datetimelike_array}
      428    0.001    0.000    0.001    0.000 config.py:589(_get_root)
      428    0.001    0.000    0.001    0.000 blocks.py:2055(check_ndim)
      428    0.000    0.000    0.001    0.000 series.py:590(name)
     1291    0.000    0.000    0.001    0.000 generic.py:43(_check)
      428    0.000    0.000    0.001    0.000 base.py:5646(_get_values_for_loc)
      428    0.000    0.000    0.002    0.000 base.py:3577(get_loc)
      428    0.000    0.000    0.003    0.000 config.py:127(_get_option)
      428    0.000    0.000    0.000    0.000 {method 'get_loc' of 'pandas._libs.index.IndexEngine' objects}
      428    0.000    0.000    0.000    0.000 generic.py:5561(__getattr__)
      428    0.000    0.000    0.001    0.000 base.py:6284(_maybe_cast_indexer)
      428    0.000    0.000    0.001    0.000 indexing.py:2481(check_deprecated_indexers)
      428    0.000    0.000    0.001    0.000 construction.py:627(_sanitize_ndim)
        1    0.000    0.000    0.000    0.000 {pandas._libs.tslibs.vectorized.ints_to_pydatetime}
      857    0.000    0.000    0.000    0.000 inference.py:321(is_hashable)
      428    0.000    0.000    0.001    0.000 construction.py:802(is_empty_data)
      428    0.000    0.000    0.001    0.000 common.py:1721(validate_all_hashable)
      860    0.000    0.000    0.000    0.000 base.py:884(__len__)
      430    0.000    0.000    0.001    0.000 {built-in method builtins.all}
      428    0.000    0.000    0.000    0.000 managers.py:1700(__init__)
      428    0.000    0.000    0.003    0.000 config.py:255(__call__)
      856    0.000    0.000    0.001    0.000 common.py:1740(<genexpr>)
      428    0.000    0.000    0.000    0.000 flags.py:47(__init__)
      428    0.000    0.000    0.000    0.000 construction.py:438(ensure_wrapped_if_datetimelike)
      428    0.000    0.000    0.001    0.000 construction.py:379(extract_array)
     1723    0.000    0.000    0.000    0.000 {built-in method builtins.getattr}
      428    0.000    0.000    0.001    0.000 series.py:687(_values)
      428    0.000    0.000    0.000    0.000 base.py:7082(maybe_extract_name)
     1284    0.000    0.000    0.000    0.000 {pandas._libs.lib.is_list_like}
      428    0.000    0.000    0.001    0.000 config.py:642(_warn_if_deprecated)
      428    0.000    0.000    0.000    0.000 series.py:542(_set_axis)
      428    0.000    0.000    0.001    0.000 common.py:552(require_length_match)
      428    0.000    0.000    0.000    0.000 construction.py:664(_sanitize_str_dtypes)
      428    0.000    0.000    0.000    0.000 base.py:2344(is_floating)
      428    0.000    0.000    0.000    0.000 construction.py:684(_maybe_repeat)
      428    0.000    0.000    0.000    0.000 config.py:630(_translate_key)
      428    0.000    0.000    0.000    0.000 common.py:160(cast_scalar_indexer)
        3    0.000    0.000    0.000    0.000 {method 'astype' of 'numpy.ndarray' objects}
      428    0.000    0.000    0.000    0.000 config.py:571(_select_options)
      428    0.000    0.000    0.000    0.000 {built-in method numpy.array}
      428    0.000    0.000    0.000    0.000 common.py:346(apply_if_callable)
      428    0.000    0.000    0.000    0.000 {method 'split' of 'str' objects}
      428    0.000    0.000    0.000    0.000 base.py:6987(ensure_index)
      428    0.000    0.000    0.000    0.000 common.py:1416(is_1d_only_ea_dtype)
      857    0.000    0.000    0.000    0.000 {built-in method builtins.hash}
      862    0.000    0.000    0.000    0.000 {built-in method builtins.issubclass}
      428    0.000    0.000    0.000    0.000 managers.py:1792(_block)
      856    0.000    0.000    0.000    0.000 {pandas._libs.lib.is_integer}
      428    0.000    0.000    0.000    0.000 {built-in method builtins.hasattr}
        1    0.000    0.000    0.001    0.001 managers.py:1601(_interleave)
      856    0.000    0.000    0.000    0.000 typing.py:1375(cast)
      428    0.000    0.000    0.000    0.000 {pandas._libs.lib.is_scalar}
        1    0.000    0.000    0.036    0.036 <string>:1(<module>)
      428    0.000    0.000    0.000    0.000 {built-in method builtins.callable}
      428    0.000    0.000    0.000    0.000 {pandas._libs.lib.is_float}
      428    0.000    0.000    0.000    0.000 {pandas._libs.algos.ensure_object}
        1    0.000    0.000    0.036    0.036 {built-in method builtins.exec}
        2    0.000    0.000    0.000    0.000 numerictypes.py:575(_can_coerce_all)
        1    0.000    0.000    0.000    0.000 {built-in method numpy.empty}
       19    0.000    0.000    0.000    0.000 numerictypes.py:584(<listcomp>)
        1    0.000    0.000    0.000    0.000 cast.py:1789(find_common_type)
        1    0.000    0.000    0.001    0.001 managers.py:1541(as_array)
        1    0.000    0.000    0.000    0.000 datetimelike.py:403(astype)
        1    0.000    0.000    0.000    0.000 managers.py:1679(<listcomp>)
        1    0.000    0.000    0.000    0.000 datetimes.py:645(astype)
        4    0.000    0.000    0.000    0.000 common.py:1274(is_bool_dtype)
        1    0.000    0.000    0.000    0.000 base.py:744(__iter__)
        2    0.000    0.000    0.000    0.000 common.py:1747(pandas_dtype)
        1    0.000    0.000    0.000    0.000 generic.py:5632(_protect_consolidate)
        1    0.000    0.000    0.000    0.000 blocks.py:1461(get_values)
        1    0.000    0.000    0.000    0.000 managers.py:1673(_consolidate_check)
        4    0.000    0.000    0.000    0.000 blocks.py:354(dtype)
        3    0.000    0.000    0.000    0.000 blocks.py:222(get_values)
        9    0.000    0.000    0.000    0.000 common.py:1552(get_dtype)
        2    0.000    0.000    0.000    0.000 base.py:53(shape)
        1    0.000    0.000    0.000    0.000 numerictypes.py:599(find_common_type)
        1    0.000    0.000    0.000    0.000 {built-in method fromkeys}
        1    0.000    0.000    0.001    0.001 frame.py:10808(values)
        3    0.000    0.000    0.000    0.000 common.py:315(is_datetime64_dtype)
        5    0.000    0.000    0.000    0.000 datetimes.py:545(dtype)
        2    0.000    0.000    0.000    0.000 common.py:581(is_dtype_equal)
        1    0.000    0.000    0.000    0.000 generic.py:5650(f)
        1    0.000    0.000    0.000    0.000 {pandas._libs.lib.dtypes_all_equal}
        1    0.000    0.000    0.000    0.000 {method 'view' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 dtypes.py:955(is_dtype)
        1    0.000    0.000    0.000    0.000 common.py:423(is_period_dtype)
        5    0.000    0.000    0.000    0.000 cast.py:1835(<genexpr>)
        2    0.000    0.000    0.000    0.000 {built-in method builtins.any}
        1    0.000    0.000    0.000    0.000 datetimelike.py:280(asi8)
        2    0.000    0.000    0.000    0.000 {method 'reshape' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 common.py:161(is_object_dtype)
        6    0.000    0.000    0.000    0.000 base.py:55(<genexpr>)
        1    0.000    0.000    0.000    0.000 {built-in method numpy.zeros}
        5    0.000    0.000    0.000    0.000 cast.py:1819(<genexpr>)
        3    0.000    0.000    0.000    0.000 cast.py:1828(<genexpr>)
        1    0.000    0.000    0.000    0.000 base.py:210(interleaved_dtype)
        1    0.000    0.000    0.000    0.000 managers.py:1665(is_consolidated)
        1    0.000    0.000    0.000    0.000 generic.py:5646(_consolidate_inplace)
        1    0.000    0.000    0.000    0.000 numerictypes.py:651(<listcomp>)
        4    0.000    0.000    0.000    0.000 blocks.py:244(mgr_locs)
        1    0.000    0.000    0.000    0.000 base.py:286(is_dtype)
        2    0.000    0.000    0.000    0.000 datetimes.py:570(tz)
        1    0.000    0.000    0.000    0.000 common.py:925(is_datetime64_ns_dtype)
        1    0.000    0.000    0.000    0.000 managers.py:618(consolidate)
        1    0.000    0.000    0.000    0.000 managers.py:1615(<listcomp>)
        1    0.000    0.000    0.000    0.000 base.py:465(find)
        1    0.000    0.000    0.000    0.000 common.py:1587(_is_dtype_type)
        3    0.000    0.000    0.000    0.000 base.py:4820(_values)
        2    0.000    0.000    0.000    0.000 cast.py:1830(<genexpr>)
        1    0.000    0.000    0.000    0.000 {method 'ravel' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 common.py:389(is_timedelta64_dtype)
        1    0.000    0.000    0.000    0.000 blocks.py:350(shape)
        1    0.000    0.000    0.000    0.000 {method 'transpose' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 managers.py:217(is_single_block)
        1    0.000    0.000    0.000    0.000 common.py:145(classes)
        1    0.000    0.000    0.000    0.000 datetimelike.py:880(freq)
        1    0.000    0.000    0.000    0.000 {built-in method numpy.asarray}
        1    0.000    0.000    0.000    0.000 common.py:147(<lambda>)
        1    0.000    0.000    0.000    0.000 {method 'keys' of 'dict' objects}
        1    0.000    0.000    0.000    0.000 numerictypes.py:652(<listcomp>)

***3 вариант***
126 function calls in 0.002 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.002    0.002    0.002    0.002 {built-in method bottleneck.reduce.nanmean}
        1    0.000    0.000    0.002    0.002 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 nanops.py:178(_has_infs)
        1    0.000    0.000    0.000    0.000 frame.py:3463(__getitem__)
        1    0.000    0.000    0.000    0.000 managers.py:1016(iget)
        1    0.000    0.000    0.000    0.000 {method 'reduce' of 'numpy.ufunc' objects}
        4    0.000    0.000    0.000    0.000 _ufunc_config.py:32(seterr)
        1    0.000    0.000    0.002    0.002 series.py:4435(_reduce)
        1    0.000    0.000    0.002    0.002 nanops.py:83(_f)
        1    0.000    0.000    0.002    0.002 nanops.py:119(f)
        4    0.000    0.000    0.000    0.000 _ufunc_config.py:131(geterr)
        1    0.000    0.000    0.002    0.002 generic.py:10594(_stat_function)
        1    0.000    0.000    0.000    0.000 frame.py:3923(_get_item_cache)
        1    0.000    0.000    0.000    0.000 generic.py:5517(__finalize__)
        2    0.000    0.000    0.000    0.000 base.py:5023(__getitem__)
        1    0.000    0.000    0.000    0.000 generic.py:239(__init__)
        2    0.000    0.000    0.000    0.000 generic.py:5577(__setattr__)
        1    0.000    0.000    0.000    0.000 frame.py:3906(_box_col_values)
        1    0.000    0.000    0.000    0.000 base.py:4973(__contains__)
        1    0.000    0.000    0.000    0.000 {method 'any' of 'numpy.generic' objects}
       11    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.000    0.000 frame.py:3411(_ixs)
        2    0.000    0.000    0.000    0.000 _ufunc_config.py:434(__exit__)
        2    0.000    0.000    0.000    0.000 nanops.py:79(check)
        1    0.000    0.000    0.000    0.000 base.py:3577(get_loc)
        1    0.000    0.000    0.002    0.002 generic.py:10679(mean)
        1    0.000    0.000    0.002    0.002 729624278.py:2(average2)
        4    0.000    0.000    0.000    0.000 {built-in method numpy.seterrobj}
        1    0.000    0.000    0.000    0.000 indexing.py:2481(check_deprecated_indexers)
        3    0.000    0.000    0.000    0.000 nanops.py:86(<genexpr>)
        1    0.000    0.000    0.000    0.000 series.py:323(__init__)
        2    0.000    0.000    0.000    0.000 _ufunc_config.py:429(__enter__)
        1    0.000    0.000    0.000    0.000 {method 'get_loc' of 'pandas._libs.index.IndexEngine' objects}
        1    0.000    0.000    0.000    0.000 common.py:161(is_object_dtype)
        1    0.000    0.000    0.000    0.000 blocks.py:358(iget)
        1    0.000    0.000    0.000    0.000 managers.py:1848(internal_values)
        3    0.000    0.000    0.000    0.000 common.py:160(cast_scalar_indexer)
        1    0.000    0.000    0.000    0.000 _methods.py:54(_any)
        1    0.000    0.000    0.000    0.000 base.py:6284(_maybe_cast_indexer)
        8    0.000    0.000    0.000    0.000 {built-in method numpy.geterrobj}
        1    0.000    0.000    0.000    0.000 nanops.py:162(_bn_ok_dtype)
        1    0.000    0.000    0.000    0.000 flags.py:47(__init__)
        1    0.000    0.000    0.002    0.002 generic.py:11099(mean)
        1    0.000    0.000    0.000    0.000 managers.py:1700(__init__)
        1    0.000    0.000    0.000    0.000 series.py:1238(_set_as_cached)
        1    0.000    0.000    0.000    0.000 common.py:346(apply_if_callable)
        1    0.000    0.000    0.000    0.000 flags.py:83(allows_duplicate_labels)
        1    0.000    0.000    0.000    0.000 inference.py:321(is_hashable)
        1    0.000    0.000    0.002    0.002 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 common.py:1148(needs_i8_conversion)
        2    0.000    0.000    0.000    0.000 _ufunc_config.py:425(__init__)
        1    0.000    0.000    0.000    0.000 common.py:1587(_is_dtype_type)
        1    0.000    0.000    0.000    0.000 _validators.py:218(validate_bool_kwarg)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.any}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.hasattr}
        1    0.000    0.000    0.000    0.000 base.py:2344(is_floating)
        1    0.000    0.000    0.000    0.000 series.py:687(_values)
        1    0.000    0.000    0.000    0.000 generic.py:546(_get_axis_number)
        1    0.000    0.000    0.000    0.000 common.py:147(<lambda>)
        3    0.000    0.000    0.000    0.000 {pandas._libs.lib.is_float}
        1    0.000    0.000    0.000    0.000 common.py:145(classes)
        1    0.000    0.000    0.000    0.000 managers.py:156(blknos)
        2    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        1    0.000    0.000    0.000    0.000 {pandas._libs.lib.item_from_zerodim}
        1    0.000    0.000    0.000    0.000 flags.py:51(allows_duplicate_labels)
        2    0.000    0.000    0.000    0.000 generic.py:349(flags)
        2    0.000    0.000    0.000    0.000 {built-in method builtins.issubclass}
        1    0.000    0.000    0.000    0.000 managers.py:1792(_block)
        2    0.000    0.000    0.000    0.000 {method 'get' of 'dict' objects}
        1    0.000    0.000    0.000    0.000 {method 'values' of 'dict' objects}
        1    0.000    0.000    0.000    0.000 function.py:49(__call__)
        1    0.000    0.000    0.000    0.000 generic.py:328(attrs)
        2    0.000    0.000    0.000    0.000 {pandas._libs.lib.is_integer}
        1    0.000    0.000    0.000    0.000 {method 'pop' of 'dict' objects}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.hash}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 typing.py:1375(cast)
        1    0.000    0.000    0.000    0.000 managers.py:172(blklocs)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.callable}
        1    0.000    0.000    0.000    0.000 {pandas._libs.lib.is_bool}
        1    0.000    0.000    0.000    0.000 {pandas._libs.lib.is_iterator}

##### 1 и 2 работают дольше из-за циклов. Соответственно, если найти решение 1б без цикла, то это её ускрорит

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [27]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

%prun get_word_reviews_count(af2010)

In [28]:
from collections import defaultdict

def get_word_reviews_count(df):
    word_reviews_count = defaultdict(list)
    
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        unique_words = set(words)  # Используем множество для поиска уникальных слов
        
        for word in unique_words:
            word_reviews_count[word].append(row['recipe_id'])
    
    return word_reviews_count

%prun get_word_reviews_count(af2010)

##### для ускорения мы уходим от словаря к автоматическому созданию пустого списка и к множеству, чтобы слова не повторялись

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [33]:
import numba
import numpy as np

In [42]:
import time

In [30]:
df=df.loc[df['rating'] > 0]

In [43]:
#A
def mape_basic(df):
    ratings = df['rating'].values
    mean_rating = sum(ratings) / len(ratings)
    mape_sum = 0
    
    for rating in ratings:
        mape_sum += abs((rating - mean_rating) / mean_rating)
    
    mape = mape_sum / len(ratings)
    return mape

%time mape_basic(df)

CPU times: total: 15.6 ms
Wall time: 18 ms


0.11138533165746958

In [44]:
#B
@numba.njit
def mape_numba(ratings):
    n = len(ratings)
    mean_rating = 0.0
    mape_sum = 0.0
    
    for i in range(n):
        rating = ratings[i]
        mean_rating += rating
    
    mean_rating /= n
    
    for i in range(n):
        rating = ratings[i]
        mape_sum += abs((rating - mean_rating) / mean_rating)
    
    mape = mape_sum / n
    return mape

%time mape_numba(df['rating'].values)

CPU times: total: 141 ms
Wall time: 126 ms


0.11138533165746958

In [47]:
#C
def mape_numpy(df):
    ratings = df['rating'].values
    mean_rating = np.mean(ratings)
    mape_sum = np.sum(np.abs((ratings - mean_rating) / mean_rating))
    mape = mape_sum / len(ratings)
    return mape

%time mape_numpy(df)

CPU times: total: 0 ns
Wall time: 0 ns


0.11138533165746735

In [46]:
#D
@numba.njit
def mape_np_numba(ratings):
    mean_rating = np.mean(ratings)
    mape_sum = 0
    
    for rating in ratings:
        mape_sum += abs((rating - mean_rating) / mean_rating)
    
    mape = mape_sum / len(ratings)
    return mape
%time mape_np_numba(df['rating'].values)

CPU times: total: 109 ms
Wall time: 112 ms


0.11138533165746958